In [1]:
!pip install transformers datasets accelerate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
from google.colab import files
import json
from datasets import Dataset

# Allow user to upload JSON file
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# Load the uploaded JSON file
with open(filename, "r") as file:
    data = json.load(file)

# Assuming the JSON has "content" fields with Jeff Wilson's responses
conversations = [item["content"] for item in data]
dataset = Dataset.from_dict({"text": conversations})

Saving cleaned_scraped_content.json to cleaned_scraped_content.json


In [4]:
def create_conversations(dataset):
    formatted_data = []
    for content in dataset["text"]:
        formatted_data.append({"prompt": "User Question:", "response": content})
    return formatted_data

formatted_dataset = create_conversations(dataset)

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

In [8]:
from transformers import TrainingArguments, Trainer

# Updated training arguments without evaluation
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",  # Disable automatic evaluation to avoid the error
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # Keep the batch size low
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True  # Mixed precision to reduce memory usage
)

from transformers import DataCollatorForLanguageModeling

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Start training without evaluation
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Step,Training Loss
10,3.759500
20,3.633100


TrainOutput(global_step=21, training_loss=3.684473310198103, metrics={'train_runtime': 13.9469, 'train_samples_per_second': 25.167, 'train_steps_per_second': 1.506, 'total_flos': 107626800384000.0, 'train_loss': 3.684473310198103, 'epoch': 2.871794871794872})

In [12]:
def chat_with_jeff(prompt):
    # Move the model to the GPU (if available)
    model.to('cuda')

    # Add some contextual leading text to improve response relevance
    context_prompt = f"Jeff Wilson, as an expert in sustainable living, {prompt}"

    # Tokenize the input and move it to the GPU as well
    inputs = tokenizer(context_prompt, return_tensors="pt").to('cuda')

    # Generate the response
    output = model.generate(**inputs, max_length=200, temperature=0.7, top_p=0.9)

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

user_input = "what is your perspective on sustainable living?"
jeff_response = chat_with_jeff(user_input)
print(jeff_response)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Jeff Wilson, as an expert in sustainable living, what is your perspective on sustainable living?


I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question. I think it's a very important question
